<a href="https://colab.research.google.com/github/alperaydyn/da.sabanciunv/blob/master/Assignment2_Solution_ALPER_AYDIN_00027137_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS404-201801

ALPER AYDIN (00027137)

# Assignment 2
## Solving Akari using CSP


### Problem Representation
Akari Puzzle, also known as Light Up, is a puzzle game developed by Nikoli in 2001. The puzzle is a board consisting of white and black cells, with some black cells having a number represents the number of light bulbs.

The game has the following rules;

1. Place light bulbs (circles) according to the following rules.
2. Light bulbs may be placed in any of the white squares, the number in the square shows how many light bulbs are next to it, vertically and horizontally.
3. Each light bulb illuminates from bulb to black square or outer frame in its row and column.
4. Every white square must be illuminated and a light bulb can not illuminate another light bulb.



Sample | Progressing | Solution | 
- | - | - | -
![Sample](http://www.nikoli.co.jp/en/puzzles/image/akari01.gif) | ![Progressing](http://www.nikoli.co.jp/en/puzzles/image/akari02.gif) | ![Solution](http://www.nikoli.co.jp/en/puzzles/image/akari03.gif) | [ ![nikoli](http://www.nikoli.co.jp/images/nikoli_logo.png)](https://www.nikoli.co.jp/en/puzzles/akari.html) |


We will be using **[Google OR Tools](https://developers.google.com/optimization/)** for solving Akari Puzzle. OR-Tools is an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming. 

It is useful to take a look at pages [Installation](https://developers.google.com/optimization/install/python/) and [Getting Started](https://developers.google.com/optimization/introduction/python) before we get started.



### Solution

#### 1. Board Display
We first define a function to display the board. We use IPython display and HTML classes and some css to decorate the results. Below are some examples of how the board is displayed.


In [32]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import copy

style = '<style> ' +\
       ' table {border-collapse:collapse;}' +\
       ' td, th{vertical-align:top important!; text-align:center !important; border:1px solid #ddd !important}' + \
       '.boardCell {text-align:center;width:25px;height:25px;font-size:8pt;vertical-align:middle;}' +\
       '.blackCell {background:#333; color:#eee;vertical-align:middle;}' +\
       '.whiteCell {background:#fff; color:#333;vertical-align:middle;}' +\
       '.bulbCell {background:#dd3; color:#dd3;vertical-align:middle;}' +\
       '.lumCell {background:#ffa; color:#ffa;vertical-align:middle;}' +\
       '</style>'
display(HTML(style))
def displayBoard(board, title):
  hb = '<table >';
  hb += '<th colspan="99">'+title+'</td>'
  for row in board:
    hb += '<tr>'
    for cell in row:
      cellClass = 'blackCell' if cell in ['0','1','2','3','4','X'] else 'bulbCell' if cell=='O' else 'lumCell' if cell=='L' else 'whiteCell'
      cellValue = ' ' if cell=='_' else cell
      hb += '<td class="boardCell '+cellClass+'">' + cellValue + '</td>'
    hb += '</tr>'
  hb += '</table>'
  return hb
  
  
board_2 = [['1','_','_','_','1','_'],['_','3','_','_','_','_',],['_','_','_','_','_','_',],['_','_','1','_','_','_',],['_','_','_','_','_','_',]]
board_1 = [['_','_','_','_','_','_','X'],['_','_','4','_','_','_','_'],['0','_','_','_','1','X','_'],['_','_','_','1','_','_','_'],['_','X','1','_','_','_','X'],['_','_','_','_','X','_','_'],['1','_','_','_','_','_','_'],]
board_0 = [['_','_','2','_','_','2','_','_'],['_','3','_','_','_','_','_','_'],['_','_','X','X','_','3','_','0'],['_','_','_','_','_','_','X','_'],['_','X','_','_','_','_','_','_'],['2','_','3','_','X','X','_','_'],['_','_','_','_','_','_','3','_'],['_','_','X','_','_','2','_','_'],]
b0 = displayBoard(board_0,'Board 0')
b1 = displayBoard(board_1,'Board 1')
b2 = displayBoard(board_2,'Board 2')
display(HTML('<table cellpadding=5 border=0><tr><td>'+b0+'</td><td>'+b1+'</td><td>'+b2+'</td></tr></table>'))
  

### 2. Board Validation

Before we construct the Solver model, we first validate the input board. The board must contain one of the following values: [ X, 0, 1, 2, 3, 4, _ ]

In [0]:
def validateBoard(board):
    invalidChars = [x for y in board for x in y if not x in ['X','0','1','2','3','4','_']]
    return len(invalidChars)==0, invalidChars

### 3. Model Introduction
Below is a simple representation of how to construct a CP Solver. 
1. **Define a class for printing the solution results.** *The solution printer is a callback defined in a Python class. You pass the callback to the solver, as shown in the next section, and the callback is executed each time a new solution is found.*
[CP Solver](https://developers.google.com/optimization/cp/cp_solver#cp-sat_solution_printer)

2. **Create the model*** from CpModel class.

3. **Create variables.** For this example, we create integer variables which can take values between 0 and 5 with the arbitrary names 'varX' and 'varY'

4. **Create constraints.** Here we create the constraint which our model shoud take care when searching a solution. We use the variables we defined in step 3 and pass to the model.Add() function as a condition parameter. We can add as many parameters as we require to the model. 
We can add conditions as simple comparisons or as function parameters or even as lambda functions. Model.Add() function only requires the parameter to be boolean condition.

5. **Solve and display.** We use CpSolver() function as solver, create a pointer to the solution printer class and call **SearchForAllSolutions()** function with the parameters model and solution printer.

In [3]:
 !pip install ortools
from ortools.sat.python import cp_model

# 1. class required for solution callback
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def OnSolutionCallback(self):
        self.__solution_count += 1
        print('Solution ',self.__solution_count)
        for v in self.__variables:
            print(v, self.Value(v))

# 2. create model
model = cp_model.CpModel()

# 3. create variables
x = model.NewIntVar(0,5,'varX')
y = model.NewIntVar(0,5,'varY')

# 4. create constraints
model.Add( x+y<7 )
model.Add( x-y>3 )
model.Add( x>2 )

model.Add( sum([x,y])>2 )

f = lambda x,y: x+y
model.Add( f(x,y)>2 )

# 5. solve and display solution
solver = cp_model.CpSolver()
solution_printer = SolutionPrinter([x,y])
status = solver.SearchForAllSolutions(model, solution_printer)

    100% |████████████████████████████████| 23.7MB 702kB/s 
Solution  1
varX 4
varY 0
Solution  2
varX 5
varY 0
Solution  3
varX 5
varY 1


### 4. Model Preperation

Since we got familiar with how the model works, we now will follow the same steps for our problem.

#### 4.1 Solution Callback
The structure is same as the previous step. Instead of directly printing the solution variables, we add a ***Value*** parameter to the **__variables** collection of the class and pass the variable values to this new parameter. We will then use the value of the variables when we display the solution. Another addition is the **disp** parameter. This is a function we pass to the SolutionPrinter function with the variables, before we call the solution. Here we define how to display the solution results.


In [0]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables, disp):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.disp = disp

    def OnSolutionCallback(self):
        self.__solution_count += 1
        for v in self.__variables:
            v.Value = self.Value(v)
        self.disp(self.__variables)

### 4.2 Create Model
We will be creating the model in the RunModel() function that we create. 

In [0]:
model = cp_model.CpModel()

### 4.3 Create Variables
In our model, variables are the **White Cells** on the board. And this cells can take values 1 (with bulb) and 0 (no bulb). So the variables will be interger values between 0 and 1. So we create NewIntVar for each white cell ('_') on the board. For doing this, we create a function that retrieves the white cells from the board.

We will be creating the variables in the RunModel() function that we create. 

In [0]:
def findWhiteCells(board):
    return [ [i,j] for i,y in enumerate(board) for j,x in enumerate(y) if x=='_']


#var = [model.NewIntVar(0, 1, 'yx%s%s'%(v[0],v[1])) for v in whiteCells]

### 4.4 Creating Constraints
This part is the most important part of the model. We have rules defined for the game and we have to formulize them as boolean conditions. So, let's do it one by one for each rule of the game.

#### 4.4.1 Number of bulb cells around a numbered cell is equal to the value of the numbered cell
First, lets visualize the phrase. 

In [7]:
display(HTML(style))
display(HTML(displayBoard([['_','1*','_'],['2*','3','3*'],['_','4*','_']],'Test')))

,1*,
2*,3,3*
,4*,


Here, the rule says that, there must be exactly 3 light bulbs around the gray cell. So cells we need to identify the coordinates of theese cells to be able to define them as variables. But before this, we need to find which cells contain number. So we create **findNumberCells()** and **findNeighbours()** functions for this purpose.

**findNumberCells** function simply loops through each cell of the board and filters the cell with a list 0-4 using a list comprehension.

**findNeighbors** function uses a matrix to calculate the coordinate of a given cell in all four directions up,down,left and right, and filter the calculations if they are inside the boundaries of the board. And finally, returns the coordinates or the indices of the found cells in the whiteCells list according to the **returnIndex** parameter.



In [0]:
def findNumberCells(board):
    nc = [ [i,j] for i,y in enumerate(board) for j,x in enumerate(y) if x in ['0','1','2','3','4']]
    return nc 

def findNeighbours(s, board, returnIndex):
    d = [[-1,0],[1,0],[0,-1],[0,1]]
    neigbours = [ list(sp) for sp in np.add(s,d) if 0<=sp[0]<len(board) and 0<=sp[1]<len(board[0])]
    return [whiteCells.index(n) for n in neigbours if n in whiteCells] if returnIndex else neigbours

We will then pass these variables as conditions in the **CreateConditions()** function.

#### 4.4.2. each white cell must be illuminated
This condition is a little bit tricky. We must first define what **illuminated** means. Let's again explain it by visualizing. Below is a sample board. There are bulbs at cells [y:0, x:0] and [y:2, x:3] with dark yellow. These bulbs illuminates the cells in vertical and horizontal directions until a black cell or a border is faced. So, **illuminated** means, for any cell, there should be at least 1 bulb in vertical or horizontal direction, but not exeed 2. Because, rule #4 says that a bulb cannot illuminate another light bulb. So there must be at much two bulbs in the union of two directions. 

If we want to formulate the condition for cell [0,0]:

**vertical: v=** [0,1],[0,2],[0,3]

**horizontal: h=** [1,0],[2,0]

v + h >= 1 and v + h <=2

In [9]:
display(HTML(style))
display(HTML(displayBoard([['O','L','L','X','_'],['L','_','X','L','X'],['L','X','L','O','L']],'illumination')))

O,L,L,X,
L,,X,L,X
L,X,L,O,L


So, we need to make a list for every white cell containing all the horizontal and vertical cells. To do this we define two functions: **findVerticalCells()** and **findHorizontalCells()**. These functions loops through each cell starting from the selected cell until a border or a balck cell and returns the coordinates or the indices of the accumulated cells according to the returnIndex parameter.

In [0]:
def findVerticalCells(s, board, returnIndex):
    vertical = []
    x = s[1]
    for y in range(s[0], -1, -1):
        if board[y][x]!='_': break
        if [y,x] not in vertical: vertical.append([y,x])
    for y in range(s[0], len(board)):
        if board[y][x]!='_': break
        if [y,x] not in vertical: vertical.append([y,x])
    return [whiteCells.index(n) for n in vertical] if returnIndex else vertical

def findHorizontalCells(s, board, returnIndex):
    horizontal = []
    y = s[0]
    for x in range(s[1], -1, -1):
        if board[y][x]!='_': break
        if [y,x] not in horizontal: horizontal.append([y,x])
    for x in range(s[1], len(board[0])):
        if board[y][x]!='_': break
        if [y,x] not in horizontal: horizontal.append([y,x])
    return [whiteCells.index(n) for n in horizontal] if returnIndex else horizontal

#### 4.4.3 No light bulbs should illuminate each other
In the previous item, we considered the case of light bulbs not exceeding two for the union of two directions. Bu this is not enough to make sure that there are no light bulbs illuminating each other, we add conditions the check this.



There is one addition helper function that is needed for passing the parameters to the model. As we had seen in the model introduction section, we add constaints as conditions represented by variables.

`model.Add( x + y < 5)`

But when we have list of variables, it is not possible to pass them one by one. So, to achieve this problem, we define a function **sumVars()** accepting a list that points to the indices of the actual variables in the **var** list and return a simple calculation of the given variables.



In [0]:
def sumVars(varList): 
    return sum([var[i] for i in varList])

By this function we can represent the sum of an unknown number of variables and convert it to a condition without passing all the variables one by one.

### 4.4.5 Creating the Constraints
Here, we are adding the constraints to the model with the help of our helper functions.

In [0]:
def createConstraints(model, board):
    constraints = []
    #1 number of bulb cells around a numbered cell is equal to the value of the numbered cell
    for numCell in findNumberCells(board):
        numCellValue = int(board[numCell[0]][numCell[1]])
        neighbours = findNeighbours(numCell, board, True)
        constraints.append( [numCell,'sum=',numCellValue ,neighbours] )
        model.Add( sumVars(neighbours)== numCellValue )
    
    #3.1 each white cell must be illuminated
    for whiteCell in findWhiteCells(board):
        h,v = findVerticalCells(whiteCell,board,True) , findHorizontalCells(whiteCell,board,True)
        constraints.append( [whiteCell,'sum>=1, sum<=2' , h+v] )
        model.Add( sumVars(h+v)>=1) # at least one bulb in h+v
        model.Add( sumVars(h+v)<=2) # at much two bulbs in h+v
        
        #3.2 no light bulbs should illuminate each other
        model.Add( sumVars(h)<=1) # horizontal neighbors can have at much one bulb 
        model.Add( sumVars(v)<=1) # vertical neighbors can have at much one bulbs 

### 4.5 Running the Model
And finally, we create a function that combines the explained tasks and run the model.

In [0]:
def runModel(board):
    global var
    global whiteCells

    boardIsValid, invalidValues = validateBoard(board)
    if not boardIsValid:
        print ('Board is not valid:',invalidValues)
        return
    
    whiteCells = findWhiteCells(board)
    
    # model    
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    
    #variables
    var = [model.NewIntVar(0, 1, 'yx%s%s'%(v[0],v[1])) for v in whiteCells]
    
    #constraints
    createConstraints(model, board)
    
    #display
    solutions = []
    disp = lambda values: solutions.append([v.Value for v in values])
    
    #solve
    solution_printer = SolutionPrinter(var,disp)
    status = solver.SearchForAllSolutions(model, solution_printer)
    
    #results
    print('Status:', [status])
    
    display(HTML(style))
    hsol = '<table><tr>'
    hsol += '<td>' + displayBoard(board,'Problem')
    sId = 0
    for solution in solutions:
        sId+=1
        hsol += '<td>'
        boards = copy.deepcopy(board)
        for i,v in enumerate(solution):
            y,x = whiteCells[i]
            if v==1 : boards[y][x] = 'O' 
        hsol += displayBoard(boards, 'Solution #'+str(sId))
    hsol += '</tr></table>'
    display(HTML(hsol))

In [28]:
# puzzle source : https://www.puzzle-light-up.com/

#EASY BOARDS
board_0 = [['1','_','_','_','1','_'],['_','3','_','_','_','_',],['_','_','_','_','_','_',],['_','_','1','_','_','_',],['_','_','_','_','_','_',]]
board_1 = [['_','_','2','_','_','2','_','_'],['_','3','_','_','_','_','_','_'],['_','_','X','X','_','3','_','0'],['_','_','_','_','_','_','X','_'],['_','X','_','_','_','_','_','_'],['2','_','3','_','X','X','_','_'],['_','_','_','_','_','_','3','_'],['_','_','X','_','_','2','_','_'],]
board_2 = [['_','_','_','_','_','_','X'],['_','_','4','_','_','_','_'],['0','_','_','_','1','X','_'],['_','_','_','1','_','_','_'],['_','X','1','_','_','_','X'],['_','_','_','_','X','_','_'],['1','_','_','_','_','_','_'],]
board_3 = [['_','_','_','_','_','_','_'],['_','1','2','_','_','X','_'],['_','_','_','_','_','1','_'],['_','_','_','0','_','_','_'],['_','X','_','_','_','_','_'],['_','0','_','_','X','3','_'],['_','_','_','_','_','_','_'],]
board_4 = [['_','_','X','X','_','_','_'],['_','_','_','_','_','_','_'],['_','_','_','_','_','_','1'],['0','_','_','0','_','_','X'],['1','_','_','_','_','_','_'],['_','_','_','_','_','_','_'],['_','_','_','X','2','_','_'],]
board_5 = [['_','_','3','_','_','X','_'],['0','_','_','_','_','_','_'],['_','_','_','X','_','_','X'],['_','_','1','X','2','_','_'],['X','_','_','1','_','_','_'],['_','_','_','_','_','_','X'],['_','X','_','_','2','_','_'],]
board_6 = [['_','_','X','_','0','_','_'],['_','_','_','_','_','_','_'],['X','_','_','_','_','_','0'],['_','_','_','1','_','_','_'],['X','_','_','_','_','_','X'],['_','_','_','_','_','_','_'],['_','_','2','_','0','_','_'],]

runModel(board_6)


Status: [2]


In [30]:
# NORMAL BOARDS
board_9 = [['_','_','_','_','_','_','0','_','_','_'],['_','0','_','X','_','_','_','_','1','_'],['_','_','_','_','0','_','1','_','_','_'],['1','_','3','_','_','_','_','_','0','_'],['_','_','_','_','_','_','_','1','_','_'],['_','_','X','_','_','_','_','_','_','_'],['_','1','_','_','_','_','_','X','_','X'],['_','_','_','2','_','1','_','_','_','_'],['_','1','_','_','_','_','X','_','3','_'],['_','_','_','0','_','_','_','_','_','_'],]
board_10= [['_','3','_','X','_','_','_','_','_','_'],['_','_','X','_','_','_','_','3','_','X'],['_','2','_','1','_','_','_','_','3','_'],['_','_','_','_','_','_','_','2','_','2'],['_','_','_','_','_','_','_','_','_','_'],['_','_','_','_','_','_','_','_','_','_'],['2','_','X','_','_','_','_','_','_','_'],['_','3','_','_','_','_','X','_','2','_'],['X','_','X','_','_','_','_','3','_','_'],['_','_','_','_','_','_','3','_','X','_'],]
runModel(board_10)

Status: [2]


In [29]:
# HARD BOARDS

board_7 = [['X','1','_','X','_','_','_','_','X','1','_','_','_','1'],['_','_','X','_','_','X','_','2','_','_','_','_','_','X'],['_','_','_','_','_','_','_','_','_','_','_','_','X','_'],['_','_','_','_','_','_','_','_','1','0','_','_','_','X'],['0','_','_','X','_','_','3','_','_','_','_','_','_','_'],['0','_','_','X','_','_','_','X','_','_','_','_','1','_'],['_','1','_','_','_','X','_','_','_','1','_','_','_','_'],['_','_','_','_','2','_','_','_','2','_','_','_','0','_'],['_','2','_','_','_','_','0','_','_','_','1','_','_','X'],['_','_','_','_','_','_','_','1','_','_','X','_','_','1'],['1','_','_','_','X','X','_','_','_','_','_','_','_','_'],['_','3','_','_','_','_','_','_','_','_','_','_','_','_'],['X','_','_','_','_','_','1','_','2','_','_','X','_','_'],['X','_','_','_','X','X','_','_','_','_','X','_','2','X'],]
board_8 = [['_','_','1','_','_','_','_'],['_','_','_','_','2','_','_'],['_','3','_','_','_','_','0'],['_','_','_','_','_','_','_'],['1','_','_','_','_','3','_'],['_','_','X','_','_','_','_'],['_','_','_','_','0','_','_'],]
runModel(board_7)

Status: [2]
